# BA - Psychologie 

Goal: The first step will be to describe a general generative process that describes whether a person succeeds on a task or not and do inference on it given fake data 


In [4]:
import torch
import pyro


In [15]:
def task():
    '''
    Simple generative process off whether someone fails a task 
    or succeds
    '''
    luck_skill = torch.distributions.uniform.Uniform(0,1).sample() ## how much luck/skill is needed for the task
    difficulty = torch.distributions.Normal(0,1).sample()
    task_importance = torch.distributions.Bernoulli(0.5).sample() # high or low 
    choice = torch.distributions.Bernoulli(0.5).sample()
    motivation = torch.distributions.Normal(0,1).sample()
    self_esteem = torch.distributions.Normal(0,1).sample()
    effort = task_importance + motivation * choice + self_esteem
    success = (difficulty < effort) if torch.distributions.Bernoulli(luck_skill).sample() else torch.distributions.Bernoulli(luck_skill).sample()
    return int(success.item())



In [16]:
results = []
for _ in range(10):
    results.append(task()) 
print(results)

[1, 1, 1, 0, 0, 1, 0, 0, 1, 1]


In [25]:
### Ok now we want to have observations of certain variables 

def task_pyro():
    luck_skill = pyro.sample('luck', pyro.distributions.Uniform(0,1))
    difficulty = pyro.sample('difficulty', pyro.distributions.Normal(0,1))
    task_importance = pyro.sample('importance', pyro.distributions.Bernoulli(0.5))
    choice = pyro.sample('choice',pyro.distributions.Bernoulli(0.5))
    motivation = pyro.sample('motivation', pyro.distributions.Normal(0,1))
    self_esteem = pyro.sample('self-esteem', pyro.distributions.Normal(0,1))
    effort = task_importance + motivation * choice + self_esteem
    decision = pyro.sample('decision',pyro.distributions.Bernoulli(luck_skill))
    luck_result = pyro.sample('luck_result',pyro.distributions.Bernoulli(0.5))
    success =  (difficulty<effort) if decision else luck_result
    return int(success.item())


In [27]:
result = []
for _ in range(10):
    result.append(task_pyro())
print(result)

[1, 1, 0, 0, 1, 0, 1, 0, 0, 0]


### Conditioning our task model on the observed values 

What is an observed variable in our model, in terms of what does the person observe ? 

* task_importance 
* choice 
* success
* motivation
* difficulty


In [28]:
obs1 = {'choice':1, 'success':0, 'motivation':1, 'task_importance':1,'difficulty':-0.5}
conditioned_task = pyro.condition(task_pyro, data = obs1)


In [ ]:
### Need to define a guide function to approximate the guide function. 
# Needs to have all the unobserved sample statementsand take the same input as the model
def guide():
    
    
    